<a href="https://colab.research.google.com/github/Jona-kh-2003/Hospital_sitemap/blob/main/Final_Hospital.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 10.6 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl siz

In [ ]:
import folium
from folium import Marker, FeatureGroup
from folium.plugins import MarkerCluster
import csv
import re
from IPython.display import HTML
from flask import Flask, render_template
import os
import gradio as gr

def load_data(file_path):
    csv_list = []
    with open(file_path, "r") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            csv_list.append(row)
    return csv_list
def create_street_map(data, selected_size=None, selected_address=None):
    # Create a street map centered at a specific location
    street_map = folium.Map(location=[17.4488, 78.3915], zoom_start=12)

    # Create feature groups for different sizes of hospitals
    big_hospitals = FeatureGroup(name='Big Hospitals')
    medium_hospitals = FeatureGroup(name='Medium Hospitals')
    small_hospitals = FeatureGroup(name='Small Hospitals')
    gachibowli_hospitals = FeatureGroup(name='Gachibowli Hospitals')
    mokila_hospitals = FeatureGroup(name='Mokila Hospitals')
    narsingi_hospitals = FeatureGroup(name='Narsingi Hospitals')

    # Add feature groups to the street map
    street_map.add_child(big_hospitals)
    street_map.add_child(medium_hospitals)
    street_map.add_child(small_hospitals)
    street_map.add_child(gachibowli_hospitals)
    street_map.add_child(mokila_hospitals)
    street_map.add_child(narsingi_hospitals)

    # Create a marker cluster for each feature group
    big_marker_cluster = MarkerCluster().add_to(big_hospitals)
    medium_marker_cluster = MarkerCluster().add_to(medium_hospitals)
    small_marker_cluster = MarkerCluster().add_to(small_hospitals)
    gachibowli_marker_cluster = MarkerCluster().add_to(gachibowli_hospitals)
    mokila_marker_cluster = MarkerCluster().add_to(mokila_hospitals)
    narsingi_marker_cluster = MarkerCluster().add_to(narsingi_hospitals)

    # Define regex patterns for size comparison and specific area identification
    big_pattern = re.compile(r'\b(big)\b', flags=re.IGNORECASE)
    medium_pattern = re.compile(r'\b(medium)\b', flags=re.IGNORECASE)
    small_pattern = re.compile(r'\b(small)\b', flags=re.IGNORECASE)
    address_pattern = re.compile(r'\b(gachibowli|mokila|narsingi)\b', flags=re.IGNORECASE)

    # Add size legend to the map
    size_legend_html = '''
    <div style="position: fixed; top: 60px; left: 20px; z-index:9999; font-size: 10px; background-color: rgba(255, 255, 255, 0.8);
    border-radius: 5px; border: 1px solid grey; padding: 5px;">
    <p><strong>Size Legend</strong></p>
    <p><i class="fa fa-circle" style="color:red"></i> Big</p>
    <p><i class="fa fa-circle" style="color:green"></i> Medium</p>
    <p><i class="fa fa-circle" style="color:blue"></i> Small</p>
    </div>
    '''
    street_map.get_root().html.add_child(folium.Element(size_legend_html))

    # Add markers for each location in the data
    for row in data:
        try:
            lat, lon = float(row[-2]), float(row[-1])
            name = row[0].strip()
            size = row[7].strip().lower()
            location_type = row[8].strip()
            address = row[1].strip().lower()  # Assuming location is stored in the 11th index

            # Check if both size and address are selected
            if selected_size and selected_address:
                # Determine marker color and size based on size using regex
                if re.search(big_pattern, size):
                    marker_color = 'red'
                    marker_radius = 10
                    Marker([lat, lon],
                           icon=folium.Icon(color=marker_color, icon='info-sign', radius=marker_radius),
                           popup=f"Name: {name}<br>Size: {size}<br>Location Type: {location_type}",
                           radius=marker_radius).add_to(big_marker_cluster)
                elif re.search(medium_pattern, size):
                    marker_color = 'green'
                    marker_radius = 7
                    Marker([lat, lon],
                           icon=folium.Icon(color=marker_color, icon='info-sign', radius=marker_radius),
                           popup=f"Name: {name}<br>Size: {size}<br>Location Type: {location_type}",
                           radius=marker_radius).add_to(medium_marker_cluster)
                elif re.search(small_pattern, size):
                    marker_color = 'blue'
                    marker_radius = 5
                    Marker([lat, lon],
                           icon=folium.Icon(color=marker_color, icon='info-sign', radius=marker_radius),
                           popup=f"Name: {name}<br>Size: {size}<br>Location Type: {location_type}",
                           radius=marker_radius).add_to(small_marker_cluster)

                # Check if the hospital address matches the selected address
                if re.search(address_pattern, address):
                    if 'gachibowli' in selected_address and 'gachibowli' in address:
                        Marker([lat, lon],
                               icon=folium.Icon(color=marker_color, icon='info-sign', radius=marker_radius),
                               popup=f"Name: {name}<br>Size: {size}<br>Location Type: {location_type}",
                               radius=marker_radius).add_to(gachibowli_marker_cluster)
                    elif 'mokila' in selected_address and 'mokila' in address:
                        Marker([lat, lon],
                               icon=folium.Icon(color=marker_color, icon='info-sign', radius=marker_radius),
                               popup=f"Name: {name}<br>Size: {size}<br>Location Type: {location_type}",
                               radius=marker_radius).add_to(mokila_marker_cluster)
                    elif 'narsingi' in selected_address and 'narsingi' in address:
                        Marker([lat, lon],
                               icon=folium.Icon(color=marker_color, icon='info-sign', radius=marker_radius),
                               popup=f"Name: {name}<br>Size: {size}<br>Location Type: {location_type}",
                               radius=marker_radius).add_to(narsingi_marker_cluster)

        except ValueError:
            continue  # Skip rows where latitude or longitude cannot be converted to float

    # Add layer control for toggling between different sizes of hospitals
    folium.LayerControl(collapsed=False).add_to(street_map)

    # Convert the street map to HTML
    html_street_map = street_map._repr_html_()

    return html_street_map


def main():
    file_paths = ['/content/Book1 - Gachibowli -Clinic.csv','/content/Book1 - Gachibowli -Poly.csv','/content/Book1 - Gachibowli-Hospitals.csv',
                  '/content/Book1 - Mokila-Clinic.csv','/content/Book1 - Mokila-Hospitals.csv','/content/Book1 - Mokila-Poly.csv',
                  '/content/Book1 - Narsingi poly .csv','/content/Book1 - Narsingi-Clinic.csv','/content/Book1 - Narsingi-hospital.csv']
    combined_data = []
    for file_path in file_paths:
        combined_data.extend(load_data(file_path))
    selected_size = ['big', 'medium', 'small']
    selected_address = ['gachibowli','mokila','narsingi']
    html_street_map = create_street_map(combined_data, selected_size, selected_address)
    display(HTML(html_street_map))
    print("Street map created successfully.")
    return html_street_map



if __name__ == "__main__":
    main()

iface = gr.Interface(
        fn=main,
        inputs=None, # Add inputs if needed
        outputs="html", # Explicitly specify the output type as HTML
        title="Street Map Generator",
        description="Generates a street map based on provided data.",
    )

    # Launch the interface
iface.launch()




Street map created successfully.
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3b7f421313c8e295c8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
